In [ ]:
pwd

### Helper Function to Test Exernal Excel Indices Before Processing (Ignore)

In [ ]:
#Helper function:
import pandas as pd

# File path and sheet name are defined outside the function as constants
file_path = 'Chelsea vs Luton Town Gathered Data.xlsx'
sheet_name = 'Pass & Interceptions'

def process_network(row_start, row_end, col_start, col_end):
    
    global file_path, sheet_name
    
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    data_slice = df.iloc[row_start-1:row_end, col_start-1:col_end]  
    
    print(data_slice)

# example call to the function with specific indices
process_network(521, 531, 19, 30)  # adjust these indices as needed

# Code Objectives:

## Graph Visualization Achieved Through:

- Detailed Passing Networks with Color-coded Weighted Edges
- Largest Strongly Connected Component showing Team Cohesion
- Eigenvector Centrality with a Color-coded Scale showing Players Involved in Critical Plays and Build-Up
- Communities (Modularity) showing which players pass to each other most frequently (Tiki-Taka Philosophy?)
- Minimum Spanning Trees to show vital linking players
- Influence Graph achieved with the help of PageRank, visually showing Player Importance in any given Interval

## Metric Calculations & Representation:

- Passes Completed (Out-Degree)
- Passes Received (In-Degree)
- Key Passes (Chance Created, Clinical Passes)
- Assists (Goal Creating Pass)

#### **Centralities**
- Degree Centrality
- Betweenness Centrality
- Closeness Centrality
- Clustering Coefficient (Average)
- Passing Network Densities




## External Data Gathering

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np



file_path = 'Chelsea vs Luton Town Gathered Data.xlsx' # Excel file path
sheet_name = 'Pass & Interceptions'
df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
graph_background = plt.imread('Soccer Pitch.jpg')
team_images = {
    'Chelsea': 'CHELSEA.png',  # Path to Chelsea's t-shirt image
    'Luton': 'LUTON.png'       # Path to Luton's t-shirt image
}

## Datasets for Field Formation Positioning and Substitutions

In [ ]:
Node_Set_All_Players_1st_Half = [
("1 Sanchez" , {"pos" : (120-10, 40)}),
("27 Gusto" , {"pos" : (120-40, 66)}),
("2 Disasi" , {"pos" : (120-25, 80-20)}),
("6 Silva" , {"pos" : (120-25, 40)}),
("26 Colwill" , {"pos" : (120-25, 80-60)}),
("25 Caicedo" , {"pos" : (120-38, 80-48)}),
("23 Gallagher" , {"pos" : (120-38, 80-32)}),
("21 Chilwell" , {"pos" : (120-40, 10)}),
("7 Sterling" , {"pos" : (120-52, 58)}) ,
("15 Jackson" , {"pos" : (120-53, 40)}) ,
("8 Fernandez" , {"pos" : (120-52, 22)}),
("24 Kaminski", {"pos": (120-110, 40)}),
("26 Giles" , {"pos": (120-90, 80-10)}),
("29 Bell" , {"pos": (120-95, 80-25)}),
("4 Lockyer" , {"pos": (120-95, 80-40)}),
("16 Burke" , {"pos": (120-95, 80-55)}),
("12 Kabore" , {"pos": (120-90, 80-70)}),
("6 Barkley", {"pos":  (120-80, 80-20)}),
("13 Nakamba", {"pos": (120-80, 80-40)}),
("14 Chong", {"pos": (120-80, 80-60)}),
("9 Morris", {"pos": (120-67, 80-30)}), 
("11 Adebayo", {"pos": (120-67, 80-50)}) 
]

Chelsea_Players_1st_Half = [
("1 Sanchez" , {"pos" : (120-10, 40)}),
("27 Gusto" , {"pos" : (120-65, 80-16.0)}),
("2 Disasi" , {"pos" : (120-40, 80-20.0)}),
("6 Silva" , {"pos" : (120-40, 80-40.0)}),
("26 Colwill" , {"pos" : (120-40, 80-60.0)}),
("23 Gallagher" , {"pos" : (120-65, 80-32.0)}),
("25 Caicedo" , {"pos" : (120-65, 80-48.0)}),
("21 Chilwell" , {"pos" : (120-65, 80-64.0)}),
("7 Sterling" , {"pos" : (120-95, 80-20.0)}) ,
("15 Jackson" , {"pos" : (120-95, 80-40.0)}) ,
("8 Fernandez" , {"pos" : (120-95, 80-60.0)})   
]

Luton_Players_1st_Half = [
("24 Kaminski", {"pos": (120-110, 40)}),
("12 Kabore" , {"pos": (120-80, 80-70)}),
("29 Bell" , {"pos": (120-90, 80-25)}),
("4 Lockyer" , {"pos": (120-90, 80-40)}),
("16 Burke" , {"pos": (120-90, 80-55)}),
("26 Giles" , {"pos": (120-80, 80-10)}),
("6 Barkley", {"pos": (120-60, 80-25)}),
("13 Nakamba", {"pos": (120-60, 80-40)}),
("14 Chong", {"pos": (120-60, 80-55)}),
("9 Morris", {"pos": (120-34, 80-30)}), 
("11 Adebayo", {"pos": (120-34, 80-50)}) 
]

# teams switch sides on pitch in the 2nd half

Node_Set_All_Players_2nd_Half_Time_45_50_60_70 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (20, 15)}),
    ("2 Disasi", {"pos": (20, 30)}),
    ("6 Silva", {"pos": (20, 50)}),
    ("26 Colwill", {"pos": (20, 65)}),
    ("25 Caicedo", {"pos": (35, 40)}),
    ("7 Sterling", {"pos": (45, 14)}),
    ("23 Gallagher", {"pos": (45, 30)}),
    ("8 Fernandez", {"pos": (45, 50)}),
    ("21 Chilwell", {"pos": (45, 65)}),
    ("15 Jackson", {"pos": (57, 40)}),
    ("24 Kaminski", {"pos": (110, 40)}),
    ("26 Giles", {"pos": (90, 10)}),
    ("29 Bell", {"pos": (95, 25)}),
    ("4 Lockyer", {"pos": (95, 40)}),
    ("16 Burke", {"pos": (95, 55)}),
    ("12 Kabore", {"pos": (90, 70)}),
    ("6 Barkley", {"pos": (80, 20)}),
    ("13 Nakamba", {"pos": (80, 40)}),
    ("14 Chong", {"pos": (80, 60)}),
    ("9 Morris", {"pos": (67, 30)}),
    ("11 Adebayo", {"pos": (67, 50)})
]

Chelsea_Players_2nd_Half_Time_45_50_60_70 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (35, 15)}),
    ("2 Disasi", {"pos": (35, 30)}),
    ("6 Silva", {"pos": (35, 50)}),
    ("26 Colwill", {"pos": (35, 65)}),
    ("25 Caicedo", {"pos": (60, 40)}),
    ("7 Sterling", {"pos": (75, 15)}),
    ("23 Gallagher", {"pos": (75, 30)}),
    ("8 Fernandez", {"pos": (75, 50)}),
    ("21 Chilwell", {"pos": (75, 65)}),
    ("15 Jackson", {"pos": (95, 40)})
]

Luton_Players_2nd_Half_Time_45_50_60_70 = [
    ("24 Kaminski", {"pos": (110, 40)}),
    ("12 Kabore", {"pos": (80, 70)}),
    ("29 Bell", {"pos": (90, 25)}),
    ("4 Lockyer", {"pos": (90, 40)}),
    ("16 Burke", {"pos": (90, 55)}),
    ("26 Giles", {"pos": (80, 10)}),
    ("6 Barkley", {"pos": (60, 25)}),
    ("13 Nakamba", {"pos": (60, 40)}),
    ("14 Chong", {"pos": (60, 55)}),
    ("9 Morris", {"pos": (34, 30)}),
    ("11 Adebayo", {"pos": (34, 50)})
]

Node_Set_All_Players_2nd_Half_Time_70_80 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (20, 15)}),
    ("2 Disasi", {"pos": (20, 30)}),
    ("6 Silva", {"pos": (20, 50)}),
    ("26 Colwill", {"pos": (20, 65)}),
    ("25 Caicedo", {"pos": (35, 40)}),
    ("7 Sterling", {"pos": (45, 14)}),
    ("23 Gallagher", {"pos": (45, 30)}),
    ("8 Fernandez", {"pos": (45, 50)}),
    ("21 Chilwell", {"pos": (45, 65)}),
    ("15 Jackson", {"pos": (57, 40)}),
    ("24 Kaminski", {"pos": (110, 40)}),
    ("12 Kabore - 45 Doughty", {"pos": (120-40, 80-14)}),
    ("16 Burke - 5 Andersen", {"pos": (120-25, 80-20)}),
    ("4 Lockyer", {"pos": (120-25, 80-40)}),
    ("29 Bell", {"pos": (120-25, 80-60)}),
    ("26 Giles", {"pos": (120-40, 80-70)}),
    ("14 Chong - 7 Ogbene", {"pos": (120-52, 80-58)}),
    ("13 Nakamba", {"pos": (120-38, 80-32)}),
    ("6 Barkley - 17 Mpanzu", {"pos": (120-38, 80-48)}),
    ("11 Adebayo - 19 Brown", {"pos": (120-52, 80-22)}),
    ("9 Morris", {"pos": (120-53, 40)})
]
    
Chelsea_Players_2nd_Half_Time_70_80 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (35, 15)}),
    ("2 Disasi", {"pos": (35, 30)}),
    ("6 Silva", {"pos": (35, 50)}),
    ("26 Colwill", {"pos": (35, 65)}),
    ("25 Caicedo", {"pos": (55, 40)}),
    ("7 Sterling", {"pos": (75, 15)}),
    ("23 Gallagher", {"pos": (75, 30)}),
    ("8 Fernandez", {"pos": (75, 50)}),
    ("21 Chilwell", {"pos": (75, 65)}),
    ("15 Jackson", {"pos": (95, 40)})
]

Luton_Players_2nd_Half_Time_70_80 = [
    ("24 Kaminski", {"pos": (110, 40)}),
    ("12 Kabore - 45 Doughty", {"pos": (120-65, 64.0)}),
    ("16 Burke - 5 Andersen", {"pos": (120-40, 60.0)}),
    ("4 Lockyer", {"pos": (120-40, 40.0)}),
    ("29 Bell", {"pos": (120-40, 20.0)}),
    ("26 Giles", {"pos": (120-65, 16.0)}),
    ("14 Chong - 7 Ogbene", {"pos": (120-95, 20.0)}),
    ("13 Nakamba", {"pos": (120-65, 48.0)}),
    ("6 Barkley - 17 Mpanzu", {"pos": (120-65, 32.0)}),
    ("11 Adebayo - 19 Brown", {"pos": (120-95, 60.0)}),
    ("9 Morris", {"pos": (120-95, 40.0)})
]

Node_Set_All_Players_2nd_Half_Time_80_90 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (20, 15)}),
    ("2 Disasi", {"pos": (20, 30)}),
    ("6 Silva", {"pos": (20, 50)}),
    ("26 Colwill", {"pos": (20, 65)}),
    ("25 Caicedo", {"pos": (35, 40)}),
    ("23 Gallagher", {"pos": (45, 14)}),
    ("15 Jackson - 16 Ugochukwu", {"pos": (45, 30)}),
    ("8 Fernandez", {"pos": (45, 50)}),
    ("21 Chilwell - 29 Maatsen", {"pos": (45, 65)}),
    ("7 Sterling", {"pos": (57, 40)}),
    ("24 Kaminski", {"pos": (110, 40)}),
    ("45 Doughty", {"pos": (120-40, 80-14)}),
    ("5 Andersen", {"pos": (120-25, 80-20)}),
    ("4 Lockyer", {"pos": (120-25, 80-40)}),
    ("29 Bell", {"pos": (120-25, 80-60)}),
    ("26 Giles", {"pos": (120-40, 80-70)}),
    ("7 Ogbene", {"pos": (120-52, 80-58)}),
    ("13 Nakamba", {"pos": (120-38, 80-32)}),
    ("17 Mpanzu", {"pos": (120-38, 80-48)}),
    ("19 Brown", {"pos": (120-52, 80-22)}),
    ("9 Morris", {"pos": (120-53, 40)})
]

Chelsea_Players_2nd_Half_Time_80_90 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (35, 15)}),
    ("2 Disasi", {"pos": (35, 30)}),
    ("6 Silva", {"pos": (35, 50)}),
    ("26 Colwill", {"pos": (35, 65)}),
    ("25 Caicedo", {"pos": (55, 40)}),
    ("23 Gallagher", {"pos": (75, 15)}),
    ("15 Jackson - 16 Ugochukwu", {"pos": (75, 30)}),
    ("8 Fernandez", {"pos": (75, 50)}),
    ("21 Chilwell - 29 Maatsen", {"pos": (75, 65)}),
    ("7 Sterling", {"pos": (95, 40)})
]   

Luton_Players_2nd_Half_Time_80_90 = [
    ("24 Kaminski", {"pos": (110, 40)}),
    ("45 Doughty", {"pos": (120-65, 64.0)}),
    ("5 Andersen", {"pos": (120-40, 60.0)}),
    ("4 Lockyer", {"pos": (120-40, 40.0)}),
    ("29 Bell", {"pos": (120-40, 20.0)}),
    ("26 Giles", {"pos": (120-65, 16.0)}),
    ("7 Ogbene", {"pos": (120-95, 20.0)}),
    ("13 Nakamba", {"pos": (120-65, 48.0)}),
    ("17 Mpanzu", {"pos": (120-65, 32.0)}),
    ("19 Brown", {"pos": (120-95, 60.0)}),
    ("9 Morris", {"pos": (120-95, 40.0)})
]  

Node_Set_All_Players_2nd_Half_Time_90_94 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (20, 15)}),
    ("2 Disasi", {"pos": (20, 30)}),
    ("6 Silva", {"pos": (20, 50)}),
    ("26 Colwill", {"pos": (20, 65)}),
    ("25 Caicedo", {"pos": (35, 40)}),
    ("23 Gallagher", {"pos": (45, 14)}),
    ("16 Ugochukwu", {"pos": (45, 30)}),
    ("8 Fernandez", {"pos": (45, 50)}),
    ("29 Maatsen", {"pos": (45, 65)}),
    ("7 Sterling - 37 Burstow", {"pos": (57, 40)}),
    ("24 Kaminski", {"pos": (110, 40)}),
    ("45 Doughty", {"pos": (120-40, 80-14)}),
    ("5 Andersen", {"pos": (120-25, 80-20)}),
    ("4 Lockyer", {"pos": (120-25, 80-40)}),
    ("29 Bell", {"pos": (120-25, 80-60)}),
    ("26 Giles", {"pos": (120-40, 80-70)}),
    ("7 Ogbene", {"pos": (120-52, 80-58)}),
    ("13 Nakamba", {"pos": (120-38, 80-32)}),
    ("17 Mpanzu", {"pos": (120-38, 80-48)}),
    ("19 Brown", {"pos": (120-52, 80-22)}),
    ("9 Morris", {"pos": (120-53, 40)})
]

Chelsea_Players_2nd_Half_Time_90_94 = [
    ("1 Sanchez", {"pos": (10, 40)}),
    ("27 Gusto", {"pos": (35, 15)}),
    ("2 Disasi", {"pos": (35, 30)}),
    ("6 Silva", {"pos": (35, 50)}),
    ("26 Colwill", {"pos": (35, 65)}),
    ("25 Caicedo", {"pos": (55, 40)}),
    ("23 Gallagher", {"pos": (75, 15)}),
    ("16 Ugochukwu", {"pos": (75, 30)}),
    ("8 Fernandez", {"pos": (75, 50)}),
    ("29 Maatsen", {"pos": (75, 65)}),
    ("7 Sterling - 37 Burstow", {"pos": (95, 40)})
]      

Luton_Players_2nd_Half_Time_90_94 = [
    ("24 Kaminski", {"pos": (110, 40)}),
    ("45 Doughty", {"pos": (120-65, 64.0)}),
    ("5 Andersen", {"pos": (120-40, 60.0)}),
    ("4 Lockyer", {"pos": (120-40, 40.0)}),
    ("29 Bell", {"pos": (120-40, 20.0)}),
    ("26 Giles", {"pos": (120-65, 16.0)}),
    ("7 Ogbene", {"pos": (120-95, 20.0)}),
    ("13 Nakamba", {"pos": (120-65, 48.0)}),
    ("17 Mpanzu", {"pos": (120-65, 32.0)}),
    ("19 Brown", {"pos": (120-95, 60.0)}),
    ("9 Morris", {"pos": (120-95, 40.0)})
]  


chelsea_players = set([
    '1 Sanchez', '27 Gusto', '2 Disasi', '6 Silva', '26 Colwill',
    '23 Gallagher', '25 Caicedo', '21 Chilwell', '7 Sterling',
    '15 Jackson', '8 Fernandez', '29 Maatsen', '16 Ugochukwu', '37 Burstow',
    '15 Jackson - 16 Ugochukwu', '21 Chilwell - 29 Maatsen', '7 Sterling - 37 Burstow', 'Maatsen', 'Ugochukwu'
    
])

## Drawing Graph, Nodes & Edges Based on External and Dataset Info

In [ ]:

def add_nodes(graph, node_set):
    for node in node_set:
        graph.add_node(node[0], **node[1])

def add_edges_from_dataframe(graph, df, start_row, end_row, start_col, end_col, skip_columns=None):
    skip_columns = skip_columns or []
    for row in range(start_row, end_row):
        source = df.iat[row - 1, start_col - 2]
        for col in range(start_col, end_col):
            if col in skip_columns:
                continue
            target = df.iat[start_row - 2, col - 1]
            value = df.iat[row - 1, col - 1]
            if value and value > 0:
                
                integer_part = int(value)
                decimal_part = int((value * 10) % 10)
                
                # determining the kind of data retrieved from excel sheet
                is_key_pass = decimal_part == 1
                is_assist = decimal_part == 2
                is_tackle = decimal_part == 3
                is_block = decimal_part == 4

                
                graph.add_edge(source, target, weight=integer_part, key_pass=is_key_pass, assist=is_assist, 
                               interception=decimal_part not in [3, 4], tackle=is_tackle, block=is_block)


def draw_graph(graph, background, full_title, team_images, chelsea_players, chelsea_formation, luton_formation):
    plt.figure(figsize=(12, 7))
    ax = plt.gca()

    # background image
    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')

    # node positions from above definitions
    pos = nx.get_node_attributes(graph, 'pos')

    # different style edges for each kind of data attribute
    for source, target, data in graph.edges(data=True):
        edge_color = 'yellow'  # Default color for normal passes
        width = data['weight'] * 0.8  # Base width based on the weight attribute
        style = 'solid'  # Default style

        # differentiating edge types by color and style
        if data.get('key_pass', False):
            edge_color = 'gold'
            width = 5
        if data.get('assist', False):
            edge_color = '#00BFFF'
            width = 5
        if data.get('tackle', False):
            edge_color = 'red'
            width = 2.5
        if data.get('block', False):
            edge_color = 'white'
            width = 2.5

        nx.draw_networkx_edges(
            graph, pos, edgelist=[(source, target)], width=width,
            edge_color=edge_color, arrows=True, ax=ax
        )

    # displaying edge weights as labels on top of edges
    edge_weights = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_weights, font_color='black', ax=ax)


    # placing team-specific images at node positions and drawing player numbers and names
    for node in graph.nodes():
        image_path = team_images['Chelsea'] if node in chelsea_players else team_images['Luton']
        image = plt.imread(image_path)
        zoom_factor = 0.25 
        im = OffsetImage(image, zoom=zoom_factor)
        ab = AnnotationBbox(im, pos[node], frameon=False, box_alignment=(0.5, 0.5))
        ax.add_artist(ab)

        try:
            number, name = node.split(' ', 1)
        except ValueError:
            number = ''
            name = node

        # position of player number on image
        number_pos = (pos[node][0], pos[node][1] - zoom_factor * 4)  # Fine-tune this offset
        plt.text(number_pos[0], number_pos[1], number, fontsize=8, ha='center', va='center', color='white', fontweight='bold')

        # position player name on image
        name_pos = (pos[node][0], pos[node][1] + zoom_factor * 12)  # Fine-tune this offset
        plt.text(name_pos[0], name_pos[1], name, fontsize=10, ha='center', va='bottom', color='black', fontweight='bold')

    # setting the title and subtitles, including the team formation(s)
    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(main_title, fontsize=20, weight='bold')
    subtitle = f"Time Interval: {time_interval} Minutes | Formation: {chelsea_formation} vs {luton_formation}"
    plt.title(subtitle, fontsize=16, style='italic', pad=10)

    plt.axis('off')
    plt.show()

def manual_degree_centrality(graph):
    centrality = {}
    n = len(graph.nodes()) - 1  
    for node in graph.nodes():
        if graph.is_directed():
            # for directed graphs, consider in-degree and out-degree separately
            in_degree = graph.in_degree(node)
            out_degree = graph.out_degree(node)
            centrality[node] = (in_degree + out_degree) / (2 * n)
        else:
            centrality[node] = graph.degree(node) / n
    return centrality
    
def print_metrics(graph, title, network_type, chelsea_players):
    print(f"{title}")
    density = nx.density(graph)
    print(f"Density: {density:.4f}")

    if network_type == 'passing':
       
        passing_data = []
        total_passes_completed = 0  

        for node in graph.nodes:
            passes_received = sum(data['weight'] for _, _, data in graph.in_edges(node, data=True))
            passes_completed = sum(data['weight'] for _, _, data in graph.out_edges(node, data=True))
            key_passes = sum(1 for _, _, data in graph.out_edges(node, data=True) if data.get('key_pass', False))
            assists = sum(1 for _, _, data in graph.out_edges(node, data=True) if data.get('assist', False))

            total_passes_completed += passes_completed  
            
            passing_data.append({
                'Player': node,
                'Passes Received': passes_received,
                'Passes Completed': passes_completed,
                'Key Passes': key_passes,
                'Assists': assists
            })

        passing_df = pd.DataFrame(passing_data)
        print("\nPassing Metrics")
        print(passing_df.to_string(index=False))
        
        print(f"\nTotal Passes Completed: {total_passes_completed}")

        print("\nCentrality Metrics")
        degree_centrality = manual_degree_centrality(graph)
        betweenness_centrality = nx.betweenness_centrality(graph)
        closeness_centrality = nx.closeness_centrality(graph)
        clustering_coefficients = nx.clustering(graph)

        centrality_df = pd.DataFrame({
            'Player': graph.nodes(),
            'Degree Centrality': [degree_centrality[node] for node in graph.nodes()],
            'Betweenness Centrality': [betweenness_centrality[node] for node in graph.nodes()],
            'Closeness Centrality': [closeness_centrality[node] for node in graph.nodes()],
            'Clustering Coefficient': [clustering_coefficients[node] for node in graph.nodes()]
        })
        print(centrality_df.to_string(index=False))

        average_clustering = nx.average_clustering(graph)
        print(f"\nGlobal Average Clustering Coefficient: {average_clustering:.4f}")

    print("\n")  
    

## Algorithms for Creating Different Graph Types for Match Insights

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import networkx as nx
from collections import defaultdict
from math import sqrt

# Taken from https://www.geeksforgeeks.org/tarjan-algorithm-find-strongly-connected-components/

class TarjanSCC:
    def __init__(self, graph):
        self.graph = graph
        self.V = len(graph.nodes())
        self.index = 0
        self.stack = []
        self.indexes = {}
        self.lowlink = {}
        self.on_stack = defaultdict(bool)
        self.sccs = []

    def run(self):
        for v in self.graph.nodes():
            if v not in self.indexes:
                self.strongconnect(v)

    def strongconnect(self, v):
        self.indexes[v] = self.index
        self.lowlink[v] = self.index
        self.index += 1
        self.stack.append(v)
        self.on_stack[v] = True

        for w in self.graph.successors(v):
            if w not in self.indexes:
                self.strongconnect(w)
                self.lowlink[v] = min(self.lowlink[v], self.lowlink[w])
            elif self.on_stack[w]:
                self.lowlink[v] = min(self.lowlink[v], self.indexes[w])

        if self.lowlink[v] == self.indexes[v]:
            scc = []
            while True:
                w = self.stack.pop()
                self.on_stack[w] = False
                scc.append(w)
                if w == v:
                    break
            self.sccs.append(scc)

    def get_sccs(self):
        return self.sccs

def draw_largest_scc(graph, background, full_title, team_images, chelsea_players):
    # drawing largest SCC using Tarjan's Algorithm
    tarjan = TarjanSCC(graph)
    tarjan.run()
    largest_scc = max(tarjan.get_sccs(), key=len)
    scc_subgraph = graph.subgraph(largest_scc).copy()
    
    plt.figure(figsize=(12, 7))
    ax = plt.gca()
    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')

    pos = nx.get_node_attributes(scc_subgraph, 'pos')
    nx.draw_networkx_edges(scc_subgraph, pos, ax=ax, edge_color='black')


    for node in scc_subgraph.nodes():
        image_path = team_images['Chelsea'] if node in chelsea_players else team_images['Luton']
        image = plt.imread(image_path)
        zoom_factor = 0.25  
        im = OffsetImage(image, zoom=zoom_factor)
        ab = AnnotationBbox(im, pos[node], frameon=False, box_alignment=(0.5, 0.5))
        ax.add_artist(ab)

        number, name = node.split(' ', 1)
        plt.text(pos[node][0], pos[node][1], number, fontsize=8, ha='center', va='center', color='white', fontweight='bold')
        plt.text(pos[node][0], pos[node][1] + 5, name, fontsize=10, ha='center', va='bottom', color='black', fontweight='bold')

    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(main_title, fontsize=20, weight='bold')
    subtitle = f"Time Interval: {time_interval} Minutes"
    plt.title(subtitle, fontsize=16, style='italic', pad=10)

    plt.axis('off')
    plt.show()


# Taken and edited from https://www.geeksforgeeks.org/eigenvector-centrality-centrality-measure/

def custom_eigenvector_centrality(G, max_iter=100, tol=1e-6, nstart=None, weight='weight'):
    if len(G) == 0:
        raise nx.NetworkXException("Empty graph.")

    if nstart is None:
        x = {n: 1.0 / len(G) for n in G}
    else:
        x = nstart

    s = 1.0 / sum(x.values())
    for k in x:
        x[k] *= s

    for _ in range(max_iter):
        xlast = x
        x = {n: 0 for n in xlast}

        for n in xlast:
            for nbr in G[n]:
                x[nbr] += xlast[n] * G[n][nbr].get(weight, 1)

        norm = sqrt(sum(v ** 2 for v in x.values()))
        x = {k: v / norm for k, v in x.items()}

        err = sum(abs(x[n] - xlast[n]) for n in x)
        if err < len(G) * tol:
            return x

    raise nx.NetworkXError("eigenvector_centrality: power iteration failed to converge in %d iterations." % max_iter)

def draw_eigenvector_centrality(graph, pos, background, full_title):
    centrality = custom_eigenvector_centrality(graph)
    plt.figure(figsize=(12, 7))
    ax = plt.gca()
    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')

    node_colors = [centrality.get(node, 0) for node in graph.nodes()]
    nx.draw_networkx_nodes(graph, pos, node_color=node_colors, cmap=plt.cm.viridis, ax=ax)

    nx.draw_networkx_edges(graph, pos, ax=ax)

    for node, (x, y) in pos.items():
        plt.text(x, y, node, fontsize=12, fontweight='bold', ha='center', color='black')


    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(f'{main_title}', fontsize=20, weight='bold')
    plt.title(f"Time Interval: {time_interval} Minutes", fontsize=16, style='italic', pad=10)

    norm = plt.Normalize(vmin=min(centrality.values()), vmax=max(centrality.values()))
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Eigenvector centrality')

    plt.axis('off')
    plt.show()

# Using the inbuilt nx library greedy-community algorithm
    
def draw_communities(graph, pos, background, full_title):
    communities = nx.algorithms.community.greedy_modularity_communities(graph)
    community_map = {}
    for i, community in enumerate(communities):
        for node in community:
            community_map[node] = i
    plt.figure(figsize=(12, 7))
    ax = plt.gca()
    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')

    nx.draw_networkx_nodes(graph, pos, node_color=list(community_map.values()), cmap=plt.cm.rainbow, ax=ax)

    player_labels = {node: node for node in graph.nodes()}
    nx.draw_networkx_labels(graph, pos, labels=player_labels, font_size=12, font_weight='bold', ax=ax)

    nx.draw_networkx_edges(graph, pos, alpha=0.5, ax=ax)

    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(f'Structure of {main_title}', fontsize=20, weight='bold')
    subtitle = f"Time Interval: {time_interval} Minutes"
    plt.title(subtitle, fontsize=16, style='italic', pad=10)

    plt.axis('off')
    plt.show()



def kruskal_mst(graph):
    
    sorted_edges = sorted(graph.edges(data=True), key=lambda x: x[2]['weight'])
    
    # disjoint set
    parent = {}
    rank = {}

    # dictionary to hold the MST
    mst = []

    def find(node):
        if parent[node] != node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(node1, node2):
        root1 = find(node1)
        root2 = find(node2)
        if root1 != root2:
            # union by rank
            if rank[root1] > rank[root2]:
                parent[root2] = root1
            elif rank[root1] < rank[root2]:
                parent[root1] = root2
            else:
                parent[root2] = root1
                rank[root1] += 1

    for node in graph.nodes():
        parent[node] = node
        rank[node] = 0

    # processing each edge in order
    for u, v, data in sorted_edges:
        if find(u) != find(v):
            union(u, v)
            mst.append((u, v, data['weight']))

    mst_graph = nx.Graph()
    mst_graph.add_weighted_edges_from(mst)
    return mst_graph

def draw_mst(graph, background, full_title, team_images, chelsea_players):
    # converting to undirected graph for MST
    undirected_graph = graph.to_undirected()
    mst = kruskal_mst(undirected_graph)
    plt.figure(figsize=(12, 7))
    ax = plt.gca()
    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')
    pos = nx.get_node_attributes(graph, 'pos')

    # drawing nodes with images numbers and names
    for node in mst.nodes():
        image_path = team_images['Chelsea'] if node in chelsea_players else team_images['Luton']
        image = plt.imread(image_path)
        zoom_factor = 0.25
        im = OffsetImage(image, zoom=zoom_factor)
        ab = AnnotationBbox(im, pos[node], frameon=False)
        ax.add_artist(ab)

        # player numbers and names
        number, name = node.split(' ', 1)
        plt.text(pos[node][0], pos[node][1], number, fontsize=8, ha='center', va='center', color='white', fontweight='bold')
        plt.text(pos[node][0], pos[node][1] + 5, name, fontsize=10, ha='center', va='bottom', color='black', fontweight='bold')

    # drawing edges
    for u, v in mst.edges():
        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]], color='yellow', lw=2)

    # title and display
    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(main_title, fontsize=20, weight='bold')
    subtitle = f"Time Interval: {time_interval} Minutes"
    plt.title(subtitle, fontsize=16, style='italic', pad=10)
    plt.axis('off')
    plt.show()
    
def draw_influence_graph(graph, background, full_title, team_images, chelsea_players):
    plt.figure(figsize=(12, 7))
    ax = plt.gca()


    ax.imshow(background, extent=[0, 120, 0, 80], aspect='equal')
    pos = nx.get_node_attributes(graph, 'pos')

    # Calculating PageRank using built in nx library algorithm
    pagerank = nx.pagerank(graph)
    # Normalize PageRank for node size and font size
    min_size = 100
    max_size = 1000
    min_font_size = 8
    max_font_size = 16
    sizes = [min_size + (max_size - min_size) * pagerank[node] / max(pagerank.values()) for node in graph.nodes()]
    font_sizes = [min_font_size + (max_font_size - min_font_size) * pagerank[node] / max(pagerank.values()) for node in graph.nodes()]

    for node, size, font_size in zip(graph.nodes(), sizes, font_sizes):
        image_path = team_images['Chelsea'] if node in chelsea_players else team_images['Luton']
        image = plt.imread(image_path)
        zoom_factor = size / 2000
        im = OffsetImage(image, zoom=zoom_factor)
        ab = AnnotationBbox(im, pos[node], xycoords='data', frameon=False)
        ax.add_artist(ab)

        try:
            number, name = node.split(' ', 1)
        except ValueError:
            number = ''
            name = node

        # font size based on node influence
        number_pos = (pos[node][0], pos[node][1])
        plt.text(number_pos[0], number_pos[1], number, fontsize=font_size, ha='center', va='center', color='white', fontweight='bold')
        name_pos = (pos[node][0], pos[node][1] + size / 200)
        plt.text(name_pos[0], name_pos[1], name, fontsize=font_size, ha='center', va='bottom', color='black', fontweight='bold')

    time_interval, main_title = full_title.split(' - ', 1)
    plt.suptitle(f'{main_title} Player Influence Graph', fontsize=20, weight='bold')
    subtitle = f"Time Interval: {time_interval} Minutes"
    plt.title(subtitle, fontsize=16, style='italic', pad=10)

    plt.axis('off')
    plt.show()
    

## Functions for Processing and Displaying the Different Types of Graphs

In [ ]:

team_colors = {node[0]: 'lightblue' for node in Chelsea_Players_1st_Half} 
team_colors.update({node[0]: 'coral' for node in Luton_Players_1st_Half})  

def get_node_set(time_interval, graph_type, team=None):
    if graph_type == "passing":
        if team == "Chelsea":
            if time_interval in ["0-10", "10-20", "20-30", "30-40", "40-45"]:
                return Chelsea_Players_1st_Half
            elif time_interval in ["45-50", "50-60", "60-70"]:
                return Chelsea_Players_2nd_Half_Time_45_50_60_70
            elif time_interval == "70-80":
                return Chelsea_Players_2nd_Half_Time_70_80
            elif time_interval == "80-90":
                return Chelsea_Players_2nd_Half_Time_80_90
            elif time_interval == "90-94":
                return Chelsea_Players_2nd_Half_Time_90_94
        elif team == "Luton":
            if time_interval in ["0-10", "10-20", "20-30", "30-40", "40-45"]:
                return Luton_Players_1st_Half
            elif time_interval in ["45-50", "50-60", "60-70"]:
                return Luton_Players_2nd_Half_Time_45_50_60_70
            elif time_interval == "70-80":
                return Luton_Players_2nd_Half_Time_70_80
            elif time_interval == "80-90":
                return Luton_Players_2nd_Half_Time_80_90
            elif time_interval == "90-94":
                return Luton_Players_2nd_Half_Time_90_94
    elif graph_type in ["interception", "combined"]:
        if time_interval in ["0-10", "10-20", "20-30", "30-40", "40-45"]:
            return Node_Set_All_Players_1st_Half
        elif time_interval in ["45-50", "50-60", "60-70"]:
            return Node_Set_All_Players_2nd_Half_Time_45_50_60_70
        elif time_interval == "70-80":
            return Node_Set_All_Players_2nd_Half_Time_70_80
        elif time_interval == "80-90":
            return Node_Set_All_Players_2nd_Half_Time_80_90
        elif time_interval == "90-94":
            return Node_Set_All_Players_2nd_Half_Time_90_94
    else:
        raise ValueError("Invalid graph type or time interval")

def add_team_nodes(graph, node_set, team_set):
    for node in node_set:
        if node[0] in [n[0] for n in team_set]:
            graph.add_node(node[0], **node[1])


def process_network(time_interval, chelsea_pass_indices, luton_pass_indices, chelsea_formation, luton_formation):
    # logic to identify player datasets based on time intervals
    if time_interval in ["0-10", "10-20", "20-30", "30-40", "40-45"]:
        chelsea_set = Chelsea_Players_1st_Half
        luton_set = Luton_Players_1st_Half
    elif time_interval in ["45-50", "50-60", "60-70"]:
        chelsea_set = Chelsea_Players_2nd_Half_Time_45_50_60_70
        luton_set = Luton_Players_2nd_Half_Time_45_50_60_70
    elif time_interval == "70-80":
        chelsea_set = Chelsea_Players_2nd_Half_Time_70_80
        luton_set = Luton_Players_2nd_Half_Time_70_80
    elif time_interval == "80-90":
        chelsea_set = Chelsea_Players_2nd_Half_Time_80_90
        luton_set = Luton_Players_2nd_Half_Time_80_90
    elif time_interval == "90-94":
        chelsea_set = Chelsea_Players_2nd_Half_Time_90_94
        luton_set = Luton_Players_2nd_Half_Time_90_94
    else:
        raise ValueError("Invalid time interval")
    
    # setup for chelsea's passing network
    chelsea_node_set = get_node_set(time_interval, "passing", "Chelsea")
    chelsea_pass_network = nx.DiGraph()
    add_team_nodes(chelsea_pass_network, chelsea_node_set, chelsea_set)
    add_edges_from_dataframe(chelsea_pass_network, df, *chelsea_pass_indices)
    pos = nx.get_node_attributes(chelsea_pass_network, 'pos')
    
    # draw the graph & print metrics (centralities)
    draw_graph(chelsea_pass_network, graph_background, f"{time_interval} - Chelsea Passing Network", team_images, chelsea_players, chelsea_formation, luton_formation)
    print_metrics(chelsea_pass_network, f"{time_interval} - Chelsea Passing Network Metrics", 'passing', chelsea_players)
    
    # draw algorithm graphs for better match insights
    draw_largest_scc(chelsea_pass_network, graph_background, f"{time_interval} - Chelsea Largest SCC", team_images, chelsea_players)
    draw_eigenvector_centrality(chelsea_pass_network, pos, graph_background, f"{time_interval} - Chelsea Eigenvector Centrality")
    draw_communities(chelsea_pass_network, pos, graph_background, f"{time_interval} - Chelsea Communities")
    draw_mst(chelsea_pass_network, graph_background, f"{time_interval} - Chelsea MST", team_images, chelsea_players)
    draw_influence_graph(chelsea_pass_network, graph_background, f"{time_interval} - Chelsea", team_images, chelsea_players)
    
    # setup for luton's passing network
    luton_node_set = get_node_set(time_interval, "passing", "Luton")
    luton_pass_network = nx.DiGraph()
    add_team_nodes(luton_pass_network, luton_node_set, luton_set)
    add_edges_from_dataframe(luton_pass_network, df, *luton_pass_indices)
    pos = nx.get_node_attributes(luton_pass_network, 'pos')
    
    # draw graph & print metrics (centralities)
    draw_graph(luton_pass_network, graph_background, f"{time_interval} - Luton Passing Network", team_images, chelsea_players, chelsea_formation, luton_formation)
    print_metrics(luton_pass_network, f"{time_interval} - Luton Passing Network Metrics", 'passing', chelsea_players)

    # draw algorithm graphs for better match insights
    draw_largest_scc(luton_pass_network, graph_background, f"{time_interval} - Luton Largest SCC", team_images, chelsea_players)
    draw_eigenvector_centrality(luton_pass_network, pos, graph_background, f"{time_interval} - Luton Eigenvector Centrality")
    draw_communities(luton_pass_network, pos, graph_background, f"{time_interval} - Luton Communities")
    draw_mst(luton_pass_network, graph_background, f"{time_interval} - Luton MST", team_images, chelsea_players)
    draw_influence_graph(luton_pass_network, graph_background, f"{time_interval} - Luton", team_images, chelsea_players)
    
    
    # setup the combined passing network (both teams)
    combined_passing_network = nx.DiGraph()
    combined_node_set = get_node_set(time_interval, "combined")
    add_team_nodes(combined_passing_network, combined_node_set, chelsea_set)
    add_team_nodes(combined_passing_network, combined_node_set, luton_set)
    add_edges_from_dataframe(combined_passing_network, df, *chelsea_pass_indices)
    add_edges_from_dataframe(combined_passing_network, df, *luton_pass_indices)

    if not combined_passing_network.number_of_edges():
        raise ValueError("Combined network has no edges. Check the data and edge-adding process.")
    
    pos = nx.get_node_attributes(combined_passing_network, 'pos')
    combined_formation = f"Chelsea: {chelsea_formation} | Luton: {luton_formation}"

    # draw graph and print metrics for both teams
    draw_graph(combined_passing_network, graph_background, f"{time_interval} - Combined Passing Network", team_images, chelsea_players, chelsea_formation, luton_formation)
    print_metrics(combined_passing_network, f"{time_interval} - Combined Passing Network Metrics", 'passing', chelsea_players)
    
    # draw algorithm graphs for better match insights
    draw_largest_scc(combined_passing_network, graph_background, f"{time_interval} - Combined Passing Largest SCC", team_images, chelsea_players)
    draw_eigenvector_centrality(combined_passing_network, pos, graph_background, f"{time_interval} - Combined Passing Eigenvector Centrality")
    draw_communities(combined_passing_network, pos, graph_background, f"{time_interval} - Combined Passing Communities")
    draw_mst(combined_passing_network, graph_background, f"{time_interval} - Combined Passing MST", team_images, chelsea_players)
    draw_influence_graph(combined_passing_network, graph_background, f"{time_interval} - Combined", team_images, chelsea_players)


# Last but not least -> call the process_network function with the correct excel indices

## Output Key:

### Nodes/Vertices are represented as jerseys with player name above and player number on the jersey.

### Edges are represented with color codes and a width/thickness based on the weight (amount of passes).


- **Yellow**:              Passes 
- **Gold**:                Key Pass
- **Light Blue**:          Assist


# Final Calls to Display Graphs based on Time Interval and Excel Data Indices

In [ ]:
process_network("0-10", (31, 42, 4, 15), (31, 42, 19, 30), "3-4-3", "5-3-2")

In [ ]:
process_network("10-20", (78, 89, 4, 15), (78, 89, 19, 30), "3-4-3", "5-3-2")

In [ ]:
process_network("20-30", (125, 136, 4, 15), (125, 136, 19, 30), "3-4-3", "5-3-2")

In [ ]:
process_network("30-40", (172, 183, 4, 15), (172, 183, 19, 30), "3-4-3", "5-3-2")

In [ ]:
process_network("40-45", (219, 230, 4, 15), (219, 230, 19, 30), "3-4-3", "5-3-2") #Increment column indices by 47 each time

In [ ]:
process_network("45-50", (266, 277, 4, 15), (266, 277, 19, 30), "4-1-4-1", "5-3-2") 

In [ ]:
process_network("50-60", (313, 324, 4, 15), (313, 324, 19, 30), "4-1-4-1", "5-3-2") #Increment column indices by 47 each time

In [ ]:
process_network("60-70", (360, 371, 4, 15), (360, 371, 19, 30), "4-1-4-1", "5-3-2") #Increment column indices by 47 each time

In [ ]:
process_network("70-80", (407, 418, 4, 15), (407, 418, 19, 30), "4-1-4-1", "3-4-3") #Increment column indice

In [ ]:
process_network("80-90", (454, 465, 4, 15), (454, 465, 19, 30), "4-1-4-1", "3-4-3") #Increment column indice

In [ ]:
process_network("90-94", (501, 512, 4, 15), (501, 512, 19, 30), "4-1-4-1", "3-4-3") #Increment column indice